In [1]:
import numpy as np
from unitconv import *     # Unit conversions
from helperfuncs import *  # Other helper functions

# hyperparameters
tol=1e-5

In [2]:
# Assumptions
# Road
lane_width_m = 3.5        # width of ego lane
rad_m = 950.0             # minimum radius

# Vehicle
V_init_kmh = 130          # initial vehicle speed
accelX_mpss = -3.0        # acceleration profile
veh_trackwidth_m = 2.0    # distance from left to right wheel
veh_wheelbase_m = 4.826   # distance from front axle to rear axle of vehicle
veh_steer_ratio = 18.0    # ratio of handwheel to tire angle
veh_Cf_deg_per_G = 6.0    # lateral tire stiffness, deg slip angle per G force
ego_accelY_mpss = roundtotenth(kmh2mps(V_init_kmh)**2 / rad_m) # A_mpss = V_mps**2 / R_m

# Not-to-exceed values
NTE_accelY_mpss = 2.0     # NOT TO EXCEED value of lateral acceleration in AD mode
margin_to_lanemarker_m = 0.2 #

In [3]:
#################
# System Design #
#################
# Enter values below to design your system.
PERCEPTION_m_ASIL_D = 250.0    # guaranteed obj det and lane marker range

In [4]:
def calc_stoppingdistanceandtime(V_init_mps, accelX_mpss, _mpsss=-5.0, dt_s=0.01):
    """
    Calculate distance to stop.
    Inputs:
         V_init_mps: initial vehicle speed in m/s
        accelX_mpss: maximum target acceleration in m/s^2 (deceleration = negative signed value)
             _mpsss: gradient of deceleration in  m/s^3.  (Default: -5 m/s^3)
    Outputs:
             Time_s: total time to decelerate to 0.0 m/s in seconds
         Distance_m: total distance required to decelerate to 0.0 m/s in meters
    """
    Distance_m = 0.0
    Time_s = 0.0
    Velo_mps = float(V_init_mps)
    CurrentDecel_mpss = 0.0

    while (Velo_mps > 0.0+tol):
        if (CurrentDecel_mpss > accelX_mpss):
            CurrentDecel_mpss += _mpsss
        Time_s += dt_s
        Distance_m += (Velo_mps * dt_s) + 1/2*(CurrentDecel_mpss * (dt_s ** 2))
        Velo_mps += CurrentDecel_mpss * dt_s

    Time_s = roundtohundredth(Time_s)
    Distance_m = roundtotenth(Distance_m)

    return Time_s, Distance_m

In [5]:
# stopping distance
Time_s, Distance_m = calc_stoppingdistanceandtime(kmh2mps(V_init_kmh),accelX_mpss)

# distance / circumference * 360_deg
traveled_deg = roundtohundredth(Distance_m/(np.pi * 2 * rad_m)*360.0)

# lane
lanemarker_movement_m = roundtotenth(rad_m * np.sin(traveled_deg*np.pi/180))
lateral_movement_m = roundtotenth(lanemarker_movement_m - (lane_width_m-veh_trackwidth_m)/2)

# kinematic tire steer angle
TSA_kine_deg = np.rad2deg(1.0/2.0*(np.arctan(veh_wheelbase_m/rad_m) + np.arctan(veh_wheelbase_m/(rad_m+veh_trackwidth_m))))

# dynamic steering angle
TSA_dyn_rad = veh_Cf_deg_per_G * ego_accelY_mpss / 9.81

# maximum steering wheel angle
SWA_deg = roundtotenth((TSA_kine_deg + TSA_dyn_rad) * veh_steer_ratio)

In [6]:
#assertions
assert (ego_accelY_mpss < NTE_accelY_mpss), 'Too fast for curve!  Your lateral acceleration, {} m/s^2, is too high.  It should not exceed {} m/s^2'.format(ego_accelY_mpss, NTE_accelY_mpss)
#assert (Distance_m < PERCEPTION_m_ASIL_D), 'Danger of over-driving your perception range because you can''t decelerate quickly enough'
assert (traveled_deg < 90.0), 'traveled_deg larger than expected.  Check your assumptions and math.'

# RQ: What is required to achieve stop in lane?
---

## Vehicle Assumptions
* Vehicle track width {{veh_trackwidth_m}} m
* Vehicle wheelbase {{veh_wheelbase_m}} m
* Ratio of handwheel to tire angle {{veh_steer_ratio}} (unitless)
* Lateral tire stiffness {{veh_Cf_deg_per_G}} deg slip angle per G force
* Vehicle deceleration {{accelX_mpss}} m/s/s

---
# Design Goals
1. When driven in AD mode, the ego vehicle shall stay in lane.
2. When driven in AD mode, the ego vehicle lateral acceleration shall not exceed {{NTE_accelY_mpss}} m/s/s.
3. The MRM shall put the vehicle in MRC as soon as possible in the ego lane.

# Scenario
The MRM begins while the AD is driving the vehicle at {{V_init_kmh}} km/h in the center of a {{lane_width_m}} m wide lane at the start of a curved road semgent with curvature {{1/rad_m}} 1/m (Radius: {{rad_m}} m).  **See open Questions

---
# Result
The above assumptions lead to the following results:

The ego vehicle would travel for {{Time_s}} sec covering {{Distance_m}} m and experience a maximum lateral acceleration of {{ego_accelY_mpss}} m/s/s.  The vehicle would travel {{traveled_deg}} degrees along the curved road segment before coming to a stop.

The lane marker will move {{lanemarker_movement_m}} m and vehicle will need to move laterally {{lateral_movement_m}} m, in order to stay in lane which will require a maximum steering angle of {{SWA_deg}} deg.

## System Requirements
* Deceleration capability greater or equal to {{accelX_mpss}} m/s/s.
* Object detection range greater or equal to {{Distance_m}} m.
* Steering capability to achieve steering angle of {{SWA_deg}} deg.

---
# Open Questions
How common are such curves?  How frequently do they occur in the ODE of our system?

---
# References
The typical highway design parameters were compiled from the following references.
<img src="files/highwaydesignREFS_radius.jpg">